# Nova Scotia Provincial Survey Control Points

**1. Import Libraries**

In [ ]:
import requests
import numpy as np
import pandas as pd

!pip install ipyleaflet
from ipyleaflet import Map, Marker

**2. Connect to Provincial Monument Dataset and Import into a Pandas DataFrame**

https://data.novascotia.ca/Lands-Forests-and-Wildlife/Nova-Scotia-Control-Monuments/jdf8-5rt8/about_data

Data Summary:

***Nova Scotia Control Monuments
Lands, Forests and Wildlife
The Nova Scotia Control Monuments (NSCM) dataset is updated and maintained from ongoing observations on survey monuments. The NSCM indicates the locations of physical survey monuments in Nova Scotia. Information includes the Station Number, Station Name, Station Description, Monument Type, Monument Status, Status Date, Construction Date, GPS suitability and general comments.***

In [ ]:

# Initialize empty Dataframe
data = pd.DataFrame()

# Intial Loop Parameters
offset = 0
response = requests.get(r"https://data.novascotia.ca/resource/jdf8-5rt8.json?$offset=" + str(offset))

# Loop through all records
while str(response.json()) != "[]":
    response = requests.get(r"https://data.novascotia.ca/resource/jdf8-5rt8.json?$offset=" + str(offset))
    temp = pd.read_json(response.text)
    data = data._append(temp, ignore_index=True)               # Append requets to Dataframe
    offset += 1000
data

**3. Query Data**

Query the Dataset for only the High Precision Network Points

In [ ]:
# Query all the High Precision Control
high_precision = data[(data['stationtyp']=="NS High Precision Network / NS Coordinate Control Station") | (data['stationtyp']=="NS High Precision Network")]
f"Number of High Precision Control Point is {high_precision.shape[0]}"

**4. Plot the High Precision Control on a Map**

In [ ]:

# Center of Nova Scotia and set zoom level to see most of the province
center = (45.318771, -63.352391)
m = Map(center=center, zoom=7)

# Loop through the Coordinates, and other attributes to create tooltip
for index, row in high_precision.iterrows():
        coords = dict(row['the_geom'])
        for k, v in coords.items():
                if v != "Point":
                        lon = v[0]
                        lat = v[1]
        marker = Marker(location=(lat, lon), title=(f"Station Number: {str(row['stationnum'])}\nStation Type: {str(row['stationtyp'])}\nMonument Type: {str(row['montypedes'])}"), draggable=False)
        m.add(marker);

display(m)